 # データの読み込み

In [10]:
import pandas as pd
import numpy as np
import os

In [3]:
INPUT_DIR = r'C:\Users\Masat\デスクトップ_Instead\webアプリ開発\Tleague_scraping'
season_year = 2019
match_2018 = pd.read_csv(os.path.join(INPUT_DIR, f'{season_year}_match_result.csv'), header=0, encoding='utf-8')
display(match_2018.head())

,db,s1,s2,s3,vm,HomeTeam_name,AwayTeam_name,db_HomeWin,s1_HomeWin,s2_HomeWin,s3_HomeWin,vm_HomeWin,HomeTeam_win,HomeTeam_rate_AAA_and_S,AwayTeam_rate_AAA_and_S,HomeTeam_rate_S,AwayTeam_rate_S
0,12,30,31,23,10,木下マイスター東京,岡山リベッツ,0,1,1,0,1,1,0.555556,0.500000,0.444444,0.083333
1,12,32,31,13,1,日本生命レッドエルフ,木下アビエル神奈川,0,1,1,0,0,0,0.636364,0.500000,0.272727,0.200000
2,20,32,23,32,0,T.T彩たま,琉球アスティーダ,1,1,0,1,0,1,0.454545,0.500000,0.181818,0.100000
3,2,31,30,23,10,岡山リベッツ,T.T彩たま,0,1,1,0,1,1,0.500000,0.454545,0.083333,0.181818
4,21,31,13,30,0,木下アビエル神奈川,トップおとめピンポンズ名古屋,1,1,0,1,0,1,0.500000,0.416667,0.200000,0.166667


In [11]:
def load_matchData()->pd.DataFrame:
    '''試合データをpd.DataFrameで読み込み'''

    INPUT_DIR = r'C:\Users\Masat\デスクトップ_Instead\webアプリ開発\Tleague_scraping'
    datasets_list = []

    season_years = ["2019", "2020", "2021"]
    # 各シーズンのデータを読み込み
    for i, season_year in enumerate(season_years):
        datasets_list.append(pd.read_csv(os.path.join(INPUT_DIR, f'{season_year}_match_result.csv'), header=0, encoding='utf-8'))
        
        # シーズンカラムを付与する
        datasets_list[i]["season_year"] = int(season_year)

    # 縦に結合
    df_dataset = pd.concat(datasets_list)

    return df_dataset

df = load_matchData()
df.head()

,db,s1,s2,s3,vm,HomeTeam_name,AwayTeam_name,db_HomeWin,s1_HomeWin,s2_HomeWin,s3_HomeWin,vm_HomeWin,HomeTeam_win,HomeTeam_rate_AAA_and_S,AwayTeam_rate_AAA_and_S,HomeTeam_rate_S,AwayTeam_rate_S,season_year
0,12,30,31,23,10,木下マイスター東京,岡山リベッツ,0,1,1,0,1,1,0.555556,0.500000,0.444444,0.083333,2019
1,12,32,31,13,1,日本生命レッドエルフ,木下アビエル神奈川,0,1,1,0,0,0,0.636364,0.500000,0.272727,0.200000,2019
2,20,32,23,32,0,T.T彩たま,琉球アスティーダ,1,1,0,1,0,1,0.454545,0.500000,0.181818,0.100000,2019
3,2,31,30,23,10,岡山リベッツ,T.T彩たま,0,1,1,0,1,1,0.500000,0.454545,0.083333,0.181818,2019
4,21,31,13,30,0,木下アビエル神奈川,トップおとめピンポンズ名古屋,1,1,0,1,0,1,0.500000,0.416667,0.200000,0.166667,2019


# 前処理

# 回帰モデルによる因果効果の推定

In [3]:
df.columns

Index(['db', 's1', 's2', 's3', 'vm', 'HomeTeam_name', 'AwayTeam_name',
       'db_HomeWin', 's1_HomeWin', 's2_HomeWin', 's3_HomeWin', 'vm_HomeWin',
       'HomeTeam_win', 'HomeTeam_rate_AAA_and_S', 'AwayTeam_rate_AAA_and_S',
       'HomeTeam_rate_S', 'AwayTeam_rate_S', 'season_year'],
      dtype='object')

In [4]:
X_columns = ['db_HomeWin', 's1_HomeWin', 's2_HomeWin', 'HomeTeam_rate_S', 'AwayTeam_rate_S']

In [8]:
import numpy as np
import math
# シグモイド関数の定義
def sigmoid(x:float)->float:
    e = math.e
    s = 1 / (1 + e**(-x))
    return s

sigmoid(0.1)

0.52497918747894

In [13]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

result = smf.glm(formula="HomeTeam_win ~ db_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      261
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -148.11
Date:                Wed, 09 Mar 2022   Deviance:                       296.21
Time:                        13:09:52   Pearson chi2:                     265.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.9394      0.435     -2.160      0.031      -1.792      -0.087
db_HomeWin          1.9088      0.287      6.648      0.000       1.346       2.472
HomeTeam_rate_S     3.5559      1.054      3.374      0.001       1.490       5.621
AwayTeam_rate_S    -3.4833      1.060     -3.285      0.001      -5.561      -1.405
===================================================================================
"""

In [11]:
result = smf.glm(formula="HomeTeam_win ~ s1_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      261
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -151.13
Date:                Tue, 08 Mar 2022   Deviance:                       302.26
Time:                        23:05:35   Pearson chi2:                     268.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.9206      0.433     -2.127      0.033      -1.769      -0.072
s1_HomeWin          1.7802      0.281      6.342      0.000       1.230       2.330
HomeTeam_rate_S     2.7796      1.015      2.739      0.006       0.790       4.769
AwayTeam_rate_S    -2.6679      1.062     -2.513      0.012      -4.749      -0.587
===================================================================================
"""

In [12]:
result = smf.glm(formula="HomeTeam_win ~ s2_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      261
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -158.62
Date:                Tue, 08 Mar 2022   Deviance:                       317.23
Time:                        23:05:55   Pearson chi2:                     264.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.6240      0.410     -1.523      0.128      -1.427       0.179
s2_HomeWin          1.4209      0.272      5.222      0.000       0.888       1.954
HomeTeam_rate_S     2.8155      1.014      2.776      0.006       0.828       4.803
AwayTeam_rate_S    -2.9341      1.021     -2.873      0.004      -4.935      -0.933
===================================================================================
"""

In [13]:
result = smf.glm(formula="HomeTeam_win ~ s3_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      261
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -162.73
Date:                Tue, 08 Mar 2022   Deviance:                       325.46
Time:                        23:06:21   Pearson chi2:                     261.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.4921      0.406     -1.211      0.226      -1.289       0.304
s3_HomeWin          1.1983      0.272      4.399      0.000       0.664       1.732
HomeTeam_rate_S     3.5263      0.992      3.556      0.000       1.583       5.470
AwayTeam_rate_S    -4.0126      1.035     -3.878      0.000      -6.041      -1.984
===================================================================================
"""

In [14]:
result = smf.glm(formula="HomeTeam_win ~ vm_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      261
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.24
Date:                Wed, 09 Mar 2022   Deviance:                       266.49
Time:                        13:10:01   Pearson chi2:                     215.
No. Iterations:                    23                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.5162      0.417     -1.237      0.216      -1.334       0.302
vm_HomeWin         24.9658   1.72e+04      0.001      0.999   -3.37e+04    3.38e+04
HomeTeam_rate_S     3.4806      1.112      3.130      0.002       1.301       5.660
AwayTeam_rate_S    -3.3901      1.110     -3.054      0.002      -5.566      -1.214
===================================================================================
"""

In [14]:
result = smf.glm(formula="HomeTeam_win ~ db_HomeWin + s2_HomeWin + HomeTeam_rate_S + AwayTeam_rate_S", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      260
Model Family:                Binomial   Df Model:                            4
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -133.33
Date:                Tue, 08 Mar 2022   Deviance:                       266.66
Time:                        23:09:04   Pearson chi2:                     257.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.8182      0.506     -3.595      0.000      -2.810      -0.827
db_HomeWin          2.0725      0.317      6.547      0.000       1.452       2.693
s2_HomeWin          1.6239      0.314      5.166      0.000       1.008       2.240
HomeTeam_rate_S     3.1080      1.131      2.749      0.006       0.892       5.324
AwayTeam_rate_S    -2.9513      1.137     -2.596      0.009      -5.180      -0.723
===================================================================================
"""

In [15]:
result = smf.glm(formula="HomeTeam_win ~ db_HomeWin", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      263
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -157.76
Date:                Tue, 08 Mar 2022   Deviance:                       315.53
Time:                        23:12:46   Pearson chi2:                     265.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8650      0.188     -4.589      0.000      -1.234      -0.496
db_HomeWin     1.8635      0.273      6.822      0.000       1.328       2.399
==============================================================================
"""

In [16]:
result = smf.glm(formula="HomeTeam_win ~ s1_HomeWin", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      263
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -156.93
Date:                Tue, 08 Mar 2022   Deviance:                       313.85
Time:                        23:13:04   Pearson chi2:                     265.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9217      0.194     -4.742      0.000      -1.303      -0.541
s1_HomeWin     1.8957      0.274      6.922      0.000       1.359       2.433
==============================================================================
"""

In [17]:
result = smf.glm(formula="HomeTeam_win ~ db_HomeWin + s1_HomeWin", data=df, family=sm.families.Binomial()).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:           HomeTeam_win   No. Observations:                  265
Model:                            GLM   Df Residuals:                      262
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -130.74
Date:                Tue, 08 Mar 2022   Deviance:                       261.49
Time:                        23:13:56   Pearson chi2:                     266.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.1110      0.309     -6.833      0.000      -2.717      -1.505
db_HomeWin     2.1541      0.329      6.550      0.000       1.510       2.799
s1_HomeWin     2.1829      0.329      6.643      0.000       1.539       2.827
==============================================================================
"""